In [ ]:
!python -m pip install pygame

# Snake game test

In [ ]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH = 800
HEIGHT = 600
GRID_SIZE = 20
GRID_WIDTH = WIDTH // GRID_SIZE
GRID_HEIGHT = HEIGHT // GRID_SIZE

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Create the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")

# Clock to control game speed
clock = pygame.time.Clock()

class Snake:
    def __init__(self):
        self.body = [(GRID_WIDTH // 2, GRID_HEIGHT // 2)]
        self.direction = (1, 0)
        self.grow_to = 0

    def move(self):
        # Calculate new head position
        head = self.body[0]
        new_head = (head[0] + self.direction[0], head[1] + self.direction[1])
        
        # Insert new head at the beginning
        self.body.insert(0, new_head)
        
        # Remove tail if not growing
        if self.grow_to > 0:
            self.grow_to -= 1
        else:
            self.body.pop()

    def grow(self):
        self.grow_to += 1

    def check_collision(self):
        # Check wall collision
        head = self.body[0]
        if (head[0] < 0 or head[0] >= GRID_WIDTH or 
            head[1] < 0 or head[1] >= GRID_HEIGHT):
            return True
        
        # Check self collision
        if head in self.body[1:]:
            return True
        
        return False

class Food:
    def __init__(self, snake):
        self.position = self.generate_position(snake)

    def generate_position(self, snake):
        while True:
            position = (
                random.randint(0, GRID_WIDTH - 1),
                random.randint(0, GRID_HEIGHT - 1)
            )
            if position not in snake.body:
                return position

def draw_grid():
    for x in range(0, WIDTH, GRID_SIZE):
        pygame.draw.line(screen, WHITE, (x, 0), (x, HEIGHT))
    for y in range(0, HEIGHT, GRID_SIZE):
        pygame.draw.line(screen, WHITE, (0, y), (WIDTH, y))

def draw_snake(snake):
    for segment in snake.body:
        rect = pygame.Rect(
            segment[0] * GRID_SIZE, 
            segment[1] * GRID_SIZE, 
            GRID_SIZE, 
            GRID_SIZE
        )
        pygame.draw.rect(screen, GREEN, rect)

def draw_food(food):
    rect = pygame.Rect(
        food.position[0] * GRID_SIZE, 
        food.position[1] * GRID_SIZE, 
        GRID_SIZE, 
        GRID_SIZE
    )
    pygame.draw.rect(screen, RED, rect)

def main():
    # Game initialization
    snake = Snake()
    food = Food(snake)
    score = 0
    
    # Font for score
    font = pygame.font.Font(None, 36)

    # Game loop
    running = True
    while running:
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            # Handle key presses for direction
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP and snake.direction != (0, 1):
                    snake.direction = (0, -1)
                elif event.key == pygame.K_DOWN and snake.direction

# Game of life

In [1]:
import numpy as np
import pygame
import time

class GameOfLife:
    def __init__(self, width=800, height=600, cell_size=10):
        """
        Initialize the Game of Life simulation
        
        Args:
            width (int): Width of the screen
            height (int): Height of the screen
            cell_size (int): Size of each cell
        """
        # Screen and cell dimensions
        self.width = width
        self.height = height
        self.cell_size = cell_size
        
        # Calculate grid dimensions
        self.grid_width = width // cell_size
        self.grid_height = height // cell_size
        
        # Initialize the grid
        self.grid = np.zeros((self.grid_height, self.grid_width), dtype=np.uint8)
        
        # Pygame setup
        pygame.init()
        self.screen = pygame.display.set_mode((width, height))
        pygame.display.set_caption("Conway's Game of Life")
        self.clock = pygame.time.Clock()
        
        # Colors
        self.BLACK = (0, 0, 0)
        self.WHITE = (255, 255, 255)
        self.GRAY = (50, 50, 50)
        
    def random_grid(self, probability=0.25):
        """
        Generate a random initial grid
        
        Args:
            probability (float): Probability of a cell being alive
        """
        self.grid = np.random.choice([0, 1], 
                                     size=(self.grid_height, self.grid_width), 
                                     p=[1-probability, probability])
    
    def count_neighbors(self, x, y):
        """
        Count live neighbors for a given cell
        
        Args:
            x (int): x-coordinate of the cell
            y (int): y-coordinate of the cell
        
        Returns:
            int: Number of live neighbors
        """
        # Create a slice around the cell, handling edge cases
        neighbors = self.grid[max(0, x-1):min(x+2, self.grid_height), 
                              max(0, y-1):min(y+2, self.grid_width)]
        
        # Subtract the cell itself
        return np.sum(neighbors) - self.grid[x, y]
    
    def update(self):
        """
        Update the grid based on Game of Life rules
        """
        # Create a copy of the grid to store next generation
        new_grid = self.grid.copy()
        
        # Iterate through each cell
        for x in range(self.grid_height):
            for y in range(self.grid_width):
                # Count live neighbors
                live_neighbors = self.count_neighbors(x, y)
                
                # Apply Game of Life rules
                if self.grid[x, y] == 1:
                    # Cell is alive
                    if live_neighbors < 2 or live_neighbors > 3:
                        new_grid[x, y] = 0  # Cell dies
                else:
                    # Cell is dead
                    if live_neighbors == 3:
                        new_grid[x, y] = 1  # Cell becomes alive
        
        # Update grid
        self.grid = new_grid
    
    def draw(self):
        """
        Draw the current grid state
        """
        # Clear the screen
        self.screen.fill(self.BLACK)
        
        # Draw grid lines
        for x in range(0, self.width, self.cell_size):
            pygame.draw.line(self.screen, self.GRAY, (x, 0), (x, self.height))
        for y in range(0, self.height, self.cell_size):
            pygame.draw.line(self.screen, self.GRAY, (0, y), (self.width, y))
        
        # Draw live cells
        for x in range(self.grid_height):
            for y in range(self.grid_width):
                if self.grid[x, y] == 1:
                    rect = pygame.Rect(y * self.cell_size, 
                                       x * self.cell_size, 
                                       self.cell_size, 
                                       self.cell_size)
                    pygame.draw.rect(self.screen, self.WHITE, rect)
        
        # Update display
        pygame.display.flip()
    
    def run(self, fps=10):
        """
        Main game loop
        
        Args:
            fps (int): Frames per second
        """
        # Generate initial random grid
        self.random_grid()
        
        # Main game loop
        running = True
        while running:
            # Handle events
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                
                # Allow drawing cells manually
                if event.type == pygame.MOUSEBUTTONDOWN:
                    x, y = pygame.mouse.get_pos()
                    grid_x = y // self.cell_size
                    grid_y = x // self.cell_size
                    self.grid[grid_x, grid_y] = 1
            
            # Update and draw
            self.update()
            self.draw()
            
            # Control frame rate
            self.clock.tick(fps)
        
        # Quit Pygame
        pygame.quit()

# Run the simulation
if __name__ == "__main__":
    game = GameOfLife()
    game.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-11-04 15:51:14.267 Python[36365:2969833] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-04 15:51:14.267 Python[36365:2969833] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 